In [215]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
import glob
import re
import time
import seaborn as sns
import xgboost as xgb
import csv
import pickle
color = sns.color_palette()

In [216]:
RAW_DATA_PATH = '/kaggle/dev/sberbank-russian-housing-market-data/raw_data/'
DATA_PATH = '/kaggle/dev/sberbank-russian-housing-market-data/'
TRAIN_DATA = RAW_DATA_PATH + 'train.csv'
TEST_DATA = RAW_DATA_PATH + 'test.csv'
MACRO_DATA = RAW_DATA_PATH + 'macro.csv'
MODELS_PATH = '/kaggle/dev/ashish/sberbank-russian-housing-market/models/'
SUBMISSIONS_PATH = '/kaggle/dev/sberbank-russian-housing-market-data/submissions/'

In [217]:
missing_df = macro_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(['missing_count'], ascending=[False])
missing_df

,column_name,missing_count
90,provision_retail_space_modern_sqm,1754
89,provision_retail_space_sqm,1753
81,modern_education_share,1389
82,old_education_build_share,1389
92,theaters_viewers_per_1000_cap,1023
87,hospital_beds_available_per_cap,1023
88,hospital_bed_occupancy_per_year,1023
28,grp_growth,1023
94,museum_visitis_per_100_cap,1023
96,population_reg_sports_share,1023


In [218]:
# Prep
train_df = pd.read_csv(TRAIN_DATA, parse_dates=['timestamp'])
macro_df = pd.read_csv(MACRO_DATA, parse_dates=['timestamp'])
test_df = pd.read_csv(TEST_DATA, parse_dates=['timestamp'])

print('Train data', train_df.shape)
print('Test data', test_df.shape)
print('Macro data', macro_df.shape)

# Fix child_on_acc_pre_school column
# macro_df.loc[macro_df['child_on_acc_pre_school'] == '#!', 'child_on_acc_pre_school'] = 0 

print('Merging with macro data..')
# Merge train/test data with macro data
train_macro_df =  pd.merge(train_df, macro_df, left_on='timestamp', right_on='timestamp', how='inner')
assert(len(train_macro_df) == len(train_df))
test_macro_df =  pd.merge(test_df, macro_df, left_on='timestamp', right_on='timestamp', how='inner')
assert(len(test_macro_df) == len(test_df))

print('Train + Macro data', train_macro_df.shape)
print('Test + Macro data', test_macro_df.shape)

# Find incompatible data types between train and test data
# for i in train_macro_df.columns:
#     try:
#         if not(train_macro_df[i].dtype == test_macro_df[i].dtype):
#             print('train:', i, '-', train_macro_df[i].dtype, '; test:', i, '-', test_macro_df[i].dtype)
#     except KeyError:
#         print('KeyError- skipping for:', i)

print('Fixing dtypes..')
# Fix incompatible data types between train and test data
train_macro_df['full_sq'] = train_macro_df['full_sq'].astype('float64')
test_macro_df['floor'] = test_macro_df['floor'].astype('float64')
test_macro_df['max_floor'] = test_macro_df['max_floor'].astype('float64')
test_macro_df['material'] = test_macro_df['material'].astype('float64')
test_macro_df['num_room'] = test_macro_df['num_room'].astype('float64')

train_columns = set(train_macro_df.columns)
drop_columns = ['id', 'timestamp', 'price_doc', 'child_on_acc_pre_school']

print('Encoding columns..')
for col in drop_columns:
    if col in train_columns:
        train_columns.remove(col)

for f in train_columns:
    if train_macro_df[f].dtype=='object':
        # print('encoding column:', f, '(original dtype:', train_macro_df[f].dtype, ')')
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(set(train_macro_df[f].values) | set(test_macro_df[f].values)))
        try:
            train_macro_df[f] = lbl.transform(list(train_macro_df[f].values))
            test_macro_df[f] = lbl.transform(list(test_macro_df[f].values))
        except TypeError as err:
            print('Error encoding column: ', f, err)

def rmsle(y_predicted, y_true):
    y_pred = y_predicted
    y_label = y_true.get_label()
    temp = np.square(np.log(y_pred + 1.0) - np.log(y_label + 1.0))
    error = np.sqrt(np.mean(temp))
    error_std = np.sqrt(np.std(temp))
    print('rmsle:', error, '; std:', error_std)
    return ("", error)

X = train_macro_df[list(train_columns)]
Y = train_macro_df.price_doc.values

train_X, val_X, train_Y, val_Y = model_selection.train_test_split(X, Y, train_size=0.7, random_state=42)

print('train_X.shape', train_X.shape)
print('train_Y.shape', train_Y.shape)
print('val_X.shape', val_X.shape)
print('val_Y.shape', val_Y.shape)

Train data (30471, 292)
Test data (7662, 291)
Macro data (2484, 100)
Merging with macro data..
Train + Macro data (30471, 391)
Test + Macro data (7662, 390)
Fixing dtypes..
Encoding columns..
train_X.shape (21329, 387)
train_Y.shape (21329,)
val_X.shape (9142, 387)
val_Y.shape (9142,)


In [219]:
model = xgb.XGBRegressor(max_depth = 10,
                        gamma=0.5,
                        objective="reg:linear",
                        n_estimators=10000,
                        learning_rate=0.005,
                        nthread=12,
                        subsample=0.8,
                        colsample_bytree=0.70,
                        colsample_bylevel=0.70,
                        seed=42,
                        silent=True)

model.fit(train_X, train_Y, eval_set=[(train_X, train_Y), (val_X, val_Y)], verbose=True, eval_metric=rmsle, early_stopping_rounds=50)

model_id = "model-" + str(int(time.time()))
pickle.dump(model, open(MODELS_PATH + model_id + ".xgb", "wb"))

print("Saved model ", model_id)

rmsle: 5.27708 ; std: 2.07792
rmsle: 5.28787 ; std: 2.3889
[0]	validation_0-:5.27708	validation_1-:5.28787
Multiple eval metrics have been passed: 'validation_1-' will be used for early stopping.

Will train until validation_1- hasn't improved in 50 rounds.
rmsle: 4.57078 ; std: 1.83067
rmsle: 4.57356 ; std: 1.98832
[1]	validation_0-:4.57078	validation_1-:4.57356
rmsle: 4.16225 ; std: 1.73042
rmsle: 4.16197 ; std: 1.85141
[2]	validation_0-:4.16225	validation_1-:4.16197
rmsle: 3.87684 ; std: 1.65397
rmsle: 3.87694 ; std: 1.76254
[3]	validation_0-:3.87684	validation_1-:3.87694
rmsle: 3.65709 ; std: 1.59759
rmsle: 3.65774 ; std: 1.69466
[4]	validation_0-:3.65708	validation_1-:3.65774
rmsle: 3.47828 ; std: 1.54721
rmsle: 3.47933 ; std: 1.64172
[5]	validation_0-:3.47828	validation_1-:3.47933
rmsle: 3.32718 ; std: 1.50446
rmsle: 3.32906 ; std: 1.59437
[6]	validation_0-:3.32718	validation_1-:3.32906
rmsle: 3.19699 ; std: 1.46715
rmsle: 3.19919 ; std: 1.55361
[7]	validation_0-:3.19699	validati

rmsle: 1.16196 ; std: 0.699629
rmsle: 1.17937 ; std: 0.744928
[74]	validation_0-:1.16196	validation_1-:1.17937
rmsle: 1.15174 ; std: 0.693991
rmsle: 1.16939 ; std: 0.739711
[75]	validation_0-:1.15174	validation_1-:1.16939
rmsle: 1.14171 ; std: 0.688487
rmsle: 1.15959 ; std: 0.73478
[76]	validation_0-:1.14171	validation_1-:1.15959
rmsle: 1.13171 ; std: 0.68325
rmsle: 1.14975 ; std: 0.72996
[77]	validation_0-:1.13171	validation_1-:1.14975
rmsle: 1.12202 ; std: 0.677953
rmsle: 1.14026 ; std: 0.725195
[78]	validation_0-:1.12202	validation_1-:1.14026
rmsle: 1.1125 ; std: 0.672696
rmsle: 1.13102 ; std: 0.720622
[79]	validation_0-:1.1125	validation_1-:1.13102
rmsle: 1.1031 ; std: 0.667482
rmsle: 1.12192 ; std: 0.716041
[80]	validation_0-:1.1031	validation_1-:1.12192
rmsle: 1.09386 ; std: 0.662392
rmsle: 1.11296 ; std: 0.711631
[81]	validation_0-:1.09386	validation_1-:1.11296
rmsle: 1.08476 ; std: 0.6574
rmsle: 1.10408 ; std: 0.707272
[82]	validation_0-:1.08476	validation_1-:1.10408
rmsle: 1.0

rmsle: 0.698882 ; std: 0.494677
rmsle: 0.7341 ; std: 0.615744
[147]	validation_0-:0.698882	validation_1-:0.7341
rmsle: 0.695067 ; std: 0.494479
rmsle: 0.730578 ; std: 0.616395
[148]	validation_0-:0.695067	validation_1-:0.730578
rmsle: 0.691355 ; std: 0.494288
rmsle: 0.727138 ; std: 0.617127
[149]	validation_0-:0.691355	validation_1-:0.727138
rmsle: 0.687634 ; std: 0.494276
rmsle: 0.723708 ; std: 0.617904
[150]	validation_0-:0.687634	validation_1-:0.723708
rmsle: 0.684009 ; std: 0.494275
rmsle: 0.720336 ; std: 0.618693
[151]	validation_0-:0.684009	validation_1-:0.720336
rmsle: 0.680344 ; std: 0.494356
rmsle: 0.716904 ; std: 0.61942
[152]	validation_0-:0.680344	validation_1-:0.716904
rmsle: 0.676782 ; std: 0.494414
rmsle: 0.713613 ; std: 0.620215
[153]	validation_0-:0.676782	validation_1-:0.713613
rmsle: 0.673244 ; std: 0.494449
rmsle: 0.71039 ; std: 0.621057
[154]	validation_0-:0.673244	validation_1-:0.71039
rmsle: 0.669728 ; std: 0.49464
rmsle: 0.707134 ; std: 0.621897
[155]	validation

rmsle: 0.512415 ; std: 0.545728
rmsle: 0.566604 ; std: 0.690173
[219]	validation_0-:0.512415	validation_1-:0.566604
rmsle: 0.510743 ; std: 0.546637
rmsle: 0.565212 ; std: 0.691316
[220]	validation_0-:0.510743	validation_1-:0.565212
rmsle: 0.509072 ; std: 0.547475
rmsle: 0.563816 ; std: 0.692325
[221]	validation_0-:0.509072	validation_1-:0.563816
rmsle: 0.507414 ; std: 0.548359
rmsle: 0.56242 ; std: 0.693321
[222]	validation_0-:0.507414	validation_1-:0.56242
rmsle: 0.505772 ; std: 0.549271
rmsle: 0.561022 ; std: 0.694319
[223]	validation_0-:0.505772	validation_1-:0.561022
rmsle: 0.504156 ; std: 0.550132
rmsle: 0.559664 ; std: 0.695375
[224]	validation_0-:0.504156	validation_1-:0.559664
rmsle: 0.50259 ; std: 0.551087
rmsle: 0.558306 ; std: 0.696356
[225]	validation_0-:0.50259	validation_1-:0.558306
rmsle: 0.501015 ; std: 0.552001
rmsle: 0.556974 ; std: 0.697371
[226]	validation_0-:0.501015	validation_1-:0.556974
rmsle: 0.499453 ; std: 0.552869
rmsle: 0.555674 ; std: 0.698337
[227]	valida

rmsle: 0.428751 ; std: 0.600001
rmsle: 0.499166 ; std: 0.752599
[290]	validation_0-:0.428751	validation_1-:0.499166
rmsle: 0.428002 ; std: 0.60065
rmsle: 0.498615 ; std: 0.753352
[291]	validation_0-:0.428002	validation_1-:0.498615
rmsle: 0.427202 ; std: 0.601277
rmsle: 0.498002 ; std: 0.754061
[292]	validation_0-:0.427202	validation_1-:0.498002
rmsle: 0.426427 ; std: 0.60183
rmsle: 0.497425 ; std: 0.754727
[293]	validation_0-:0.426427	validation_1-:0.497425
rmsle: 0.425686 ; std: 0.60242
rmsle: 0.496898 ; std: 0.755431
[294]	validation_0-:0.425686	validation_1-:0.496898
rmsle: 0.424928 ; std: 0.602942
rmsle: 0.496356 ; std: 0.756106
[295]	validation_0-:0.424928	validation_1-:0.496356
rmsle: 0.424212 ; std: 0.603571
rmsle: 0.495815 ; std: 0.756802
[296]	validation_0-:0.424212	validation_1-:0.495815
rmsle: 0.423493 ; std: 0.604142
rmsle: 0.495288 ; std: 0.757472
[297]	validation_0-:0.423493	validation_1-:0.495288
rmsle: 0.422777 ; std: 0.604712
rmsle: 0.494768 ; std: 0.758165
[298]	valid

rmsle: 0.389577 ; std: 0.632234
rmsle: 0.472334 ; std: 0.793834
[362]	validation_0-:0.389577	validation_1-:0.472334
rmsle: 0.389189 ; std: 0.632478
rmsle: 0.472125 ; std: 0.794295
[363]	validation_0-:0.389189	validation_1-:0.472125
rmsle: 0.388847 ; std: 0.632815
rmsle: 0.471918 ; std: 0.794749
[364]	validation_0-:0.388847	validation_1-:0.471918
rmsle: 0.388529 ; std: 0.63321
rmsle: 0.471716 ; std: 0.795206
[365]	validation_0-:0.388529	validation_1-:0.471716
rmsle: 0.38819 ; std: 0.633565
rmsle: 0.471509 ; std: 0.795638
[366]	validation_0-:0.38819	validation_1-:0.471509
rmsle: 0.387895 ; std: 0.633997
rmsle: 0.471311 ; std: 0.796088
[367]	validation_0-:0.387895	validation_1-:0.471311
rmsle: 0.387579 ; std: 0.634362
rmsle: 0.471115 ; std: 0.796537
[368]	validation_0-:0.387579	validation_1-:0.471115
rmsle: 0.387264 ; std: 0.63473
rmsle: 0.470915 ; std: 0.79698
[369]	validation_0-:0.387264	validation_1-:0.470915
rmsle: 0.386966 ; std: 0.635093
rmsle: 0.470728 ; std: 0.797423
[370]	validat

rmsle: 0.371883 ; std: 0.650778
rmsle: 0.463107 ; std: 0.820473
[433]	validation_0-:0.371883	validation_1-:0.463107
rmsle: 0.371719 ; std: 0.650988
rmsle: 0.463031 ; std: 0.820743
[434]	validation_0-:0.371719	validation_1-:0.463031
rmsle: 0.371599 ; std: 0.651258
rmsle: 0.46297 ; std: 0.821054
[435]	validation_0-:0.371599	validation_1-:0.46297
rmsle: 0.371464 ; std: 0.651514
rmsle: 0.462913 ; std: 0.821368
[436]	validation_0-:0.371464	validation_1-:0.462913
rmsle: 0.371306 ; std: 0.651711
rmsle: 0.462844 ; std: 0.821654
[437]	validation_0-:0.371306	validation_1-:0.462844
rmsle: 0.371151 ; std: 0.651927
rmsle: 0.462788 ; std: 0.821979
[438]	validation_0-:0.371151	validation_1-:0.462788
rmsle: 0.370971 ; std: 0.652068
rmsle: 0.462727 ; std: 0.822285
[439]	validation_0-:0.370971	validation_1-:0.462727
rmsle: 0.370784 ; std: 0.652192
rmsle: 0.462666 ; std: 0.822592
[440]	validation_0-:0.370784	validation_1-:0.462666
rmsle: 0.370608 ; std: 0.652346
rmsle: 0.462616 ; std: 0.822895
[441]	vali

rmsle: 0.362245 ; std: 0.659131
rmsle: 0.46067 ; std: 0.838607
[505]	validation_0-:0.362245	validation_1-:0.46067
rmsle: 0.362137 ; std: 0.659192
rmsle: 0.460656 ; std: 0.838805
[506]	validation_0-:0.362137	validation_1-:0.460656
rmsle: 0.362002 ; std: 0.659215
rmsle: 0.460639 ; std: 0.839003
[507]	validation_0-:0.362002	validation_1-:0.460639
rmsle: 0.361922 ; std: 0.659323
rmsle: 0.460626 ; std: 0.839203
[508]	validation_0-:0.361922	validation_1-:0.460626
rmsle: 0.361822 ; std: 0.659391
rmsle: 0.460616 ; std: 0.839396
[509]	validation_0-:0.361822	validation_1-:0.460616
rmsle: 0.361729 ; std: 0.659444
rmsle: 0.460603 ; std: 0.839595
[510]	validation_0-:0.361729	validation_1-:0.460603
rmsle: 0.361628 ; std: 0.659502
rmsle: 0.460583 ; std: 0.839748
[511]	validation_0-:0.361628	validation_1-:0.460583
rmsle: 0.361541 ; std: 0.659598
rmsle: 0.460577 ; std: 0.839937
[512]	validation_0-:0.361541	validation_1-:0.460577
rmsle: 0.361428 ; std: 0.659622
rmsle: 0.46057 ; std: 0.840131
[513]	valid

rmsle: 0.356003 ; std: 0.661706
rmsle: 0.460645 ; std: 0.85059
[577]	validation_0-:0.356003	validation_1-:0.460645
rmsle: 0.355932 ; std: 0.66174
rmsle: 0.460649 ; std: 0.850698
[578]	validation_0-:0.355932	validation_1-:0.460649
rmsle: 0.355859 ; std: 0.661735
rmsle: 0.460651 ; std: 0.850818
[579]	validation_0-:0.355859	validation_1-:0.460651
rmsle: 0.355793 ; std: 0.661739
rmsle: 0.460663 ; std: 0.850944
[580]	validation_0-:0.355793	validation_1-:0.460663
rmsle: 0.355714 ; std: 0.661732
rmsle: 0.460669 ; std: 0.851072
[581]	validation_0-:0.355714	validation_1-:0.460669
rmsle: 0.355618 ; std: 0.66169
rmsle: 0.46069 ; std: 0.851233
[582]	validation_0-:0.355618	validation_1-:0.46069
rmsle: 0.355545 ; std: 0.661697
rmsle: 0.460702 ; std: 0.851378
[583]	validation_0-:0.355545	validation_1-:0.460702
rmsle: 0.355469 ; std: 0.661695
rmsle: 0.460713 ; std: 0.851506
[584]	validation_0-:0.355469	validation_1-:0.460713
rmsle: 0.355392 ; std: 0.661681
rmsle: 0.460713 ; std: 0.851619
[585]	validat

In [220]:
test_columns = list(train_columns)
test_columns.append('id')

test_X =  test_macro_df[test_columns]
print("test_X.shape", test_X.shape)

test_X.shape (7662, 388)


In [232]:
submission_columns = ['id', 'predicted_price_doc']
model = pickle.load(open(MODELS_PATH + model_id + ".xgb", 'rb'))
filename = SUBMISSIONS_PATH + 'submission-' + str(int(time.time())) + ".csv"
test_X = test_X.assign(predicted_price_doc=model.predict(test_X.loc[:, train_columns]))
predicted_submission_df = test_X[submission_columns]
predicted_submission_df.columns = ['id', 'price_doc']
predicted_submission_df.to_csv(filename, index=False)
print('Generated submission file: {}'.format(filename))

Generated submission file: /kaggle/dev/sberbank-russian-housing-market-data/submissions/submission-1495444828.csv
